# Notebook to generate time averaged Picked spectra.
# Title Notebook to generate time averaged Dipolarity.

This notebook generates time average and standard deviations of dipolarity. Input file name is defined by
[picked sph prefix] l[degree#] m[order#][c/s].dat in the control file

## Initial setup

In [1]:
import sys
import numpy as np
from ctypes import *

libname = "libcalypso_to_pythons.so"

Move current directory to directory with data to be averaged

In [2]:
cd ./

/Volumes/Sources/matsui/Kemorin_MHD/MHD/pythons


Check if shared libraly to load is there

In [3]:
ls -l "libcalypso_to_pythons.so"

-rwxr-xr-x  1 matsui  staff  787909 Jun 28 18:19 libcalypso_to_pythons.so*


Load dynamic library to run the program

In [4]:
flib = cdll.LoadLibrary("libcalypso_to_pythons.so")

## Go to data directory

Move current directory to directory with data to be averaged

In [5]:
cd  /Volumes/ThunderDock_P/matsui/sph_shell_336_2/append

/Volumes/ThunderDock_P/matsui/sph_shell_336_2/append


Check if data file to be averaged is there

In [6]:
ls -l

total 1170340
-rw-r--r--  1 matsui  staff           0 Jun 28 16:49 picked_mode.dat
-rw-------@ 1 matsui  staff  1179012782 Jun 28 15:11 picked_mode_l2_m0c.dat
-rw-------@ 1 matsui  staff     6234131 Jun 25 07:20 picked_mode_l2_m0c.dat.gz
-rw-r--r--  1 matsui  staff       14930 Jun 28 18:17 t_ave_icked_mode_l2_m0c.dat
-rw-r--r--@ 1 matsui  staff      193218 Jun 28 17:23 t_ave_picked_mode_l2_m0c.dat
-rw-r--r--  1 matsui  staff       14930 Jun 28 18:17 t_rms_icked_mode_l2_m0c.dat
-rw-r--r--  1 matsui  staff      193218 Jun 28 17:23 t_rms_picked_mode_l2_m0c.dat
-rw-r--r--  1 matsui  staff       14930 Jun 28 18:17 t_sigma_icked_mode_l2_m0c.dat
-rw-r--r--  1 matsui  staff      193218 Jun 28 17:23 t_sigma_picked_mode_l2_m0c.dat


## Time averaging for volume mean square and average

Take time average of [picked sph prefix] l[degree#] m[order#][c/s].dat


Set file name to be averaged as file_name, start time as start_time, and end time as end_time.

In [7]:
file_name = "picked_mode_l2_m0c.dat"
# Load time average and standard deviation
flib.check_picked_sph_spectr_f.restype = c_void_p
flib.check_picked_sph_spectr_f.argtypes = [c_char_p]

flib.check_picked_sph_spectr_f(file_name.encode())

 Open file: picked_mode_l2_m0c.dat
 Start step and time:          100   1.0000000000000001E-005
 End step and time:     18640500   3.6781000004807201     
 Saved hermonics mode:
           1           2           0
 Saved radial points:
           1          46  0.53418396914844402     
           2          50  0.53846153846153799     
           3          54  0.54273910777463297     
           4         141   1.5317832045039801     
           5         146   1.5384615384615401     
 Saved field names:
           1 velocity_pol
           2 velocity_tor
           3 velocity_pol_dr
           4 temperature
           5 pressure
           6 vorticity_pol
           7 vorticity_tor
           8 vorticity_pol_dr
           9 magnetic_field_pol
          10 magnetic_field_tor
          11 magnetic_field_pol_dr
          12 current_density_pol
          13 current_density_tor
          14 current_density_pol_dr
          15 grad_temp_pol
          16 grad_temp_tor
          17 grad_tem

4496399360

Start time average program. 
time average data will be "t_ave_[file_name]",
and standard deviation will be "t_sdev_[file_name]".

In [8]:
start_time = 0.5
end_time =   1.0

flib.time_ave_picked_sph_spectr_f.restype = c_int
flib.time_ave_picked_sph_spectr_f.argtypes = [c_char_p, c_double, c_double]

n = flib.time_ave_picked_sph_spectr_f(file_name.encode(), c_double(start_time), c_double(end_time))

 Open file: picked_mode_l2_m0c.dat
#
# num_layers, num_spectr
               5               1
# number of component
              46
#   Start and end time
       5.000000000129E-01       1.000000000027E+00
#
# num_layers, num_spectr
               5               1
# number of component
              46
#   Start and end time
       5.000000000129E-01       1.000000000027E+00
#
# num_layers, num_spectr
               5               1
# number of component
              46
#   Start and end time
       5.000000000129E-01       1.000000000027E+00


In [19]:
# Set component name to be plotted
field_name = ["temperature", "temperature"]
degree =    [2, 2]
order =     [0, 0]
radius_id = [50, 146]

if len(degree) != len(order):
  print("Number of harmonics degree and order does not match")
  stop
if len(radius_id) != len(degree):
  print("Number of harmonics modes and radial grids does not match")
  stop
if len(field_name) != len(degree):
  print("Number of field_name and number of harmonics modes does not match")
  stop
n_line = len(field_name)

# Allocate time series data to read
radius = np.zeros((0),dtype=np.float64)
ave = np.zeros((0),dtype=np.float64)
rms = np.zeros((0),dtype=np.float64)
sdev = np.zeros((0),dtype=np.float64)

r_in = np.zeros((1),dtype=np.float64)
ave_in = np.zeros((1),dtype=np.float64)
rms_in = np.zeros((1),dtype=np.float64)
sdev_in = np.zeros((1),dtype=np.float64)


# Find compoenent address to plot
flib.get_each_tave_picked_spectr_f.restype = c_void_p
flib.get_each_tave_picked_spectr_f.argtypes = [c_char_p, c_int, c_int, c_int,
                                          np.ctypeslib.ndpointer(dtype=np.float64),
                                          np.ctypeslib.ndpointer(dtype=np.float64),
                                          np.ctypeslib.ndpointer(dtype=np.float64),
                                          np.ctypeslib.ndpointer(dtype=np.float64)]
icou = 0
for each_field in field_name:
    print(each_field)
    flib.get_each_tave_picked_spectr_f(each_field.encode(), c_int(radius_id[icou]),
                              c_int(degree[icou]), c_int(order[icou]),
                              r_in, ave_in, rms_in, sdev_in)
    radius = np.append(radius, r_in)
    ave = np.append(ave, ave_in)
    rms = np.append(rms, rms_in)
    sdev = np.append(sdev, sdev_in)
    icou = icou+1

icou = 0
for icou in range(n_line):
    print('Field name for data ', icou, ': ',  field_name[icou])
    print('    Radius:           ', radius_id[icou], radius[icou])
    print('    Degree and order: ', degree[icou], order[icou])
    print('    Average:          ', ave[icou])
    print('    Std. deviation:   ', sdev[icou])

    print('    RMS:              ', rms[icou])


temperature
 fld_nametemperature
 get_each_picked_fld_address           4
 get_each_picked_sph_address           2
temperature
 fld_nametemperature
 get_each_picked_fld_address           4
 get_each_picked_sph_address           5
Field name for data  0 :  temperature
    Radius:            50 0.538461538461538
    Degree and order:  2 0
    Average:           0.3259570153221679
    Dtd. deviation:    0.06832503083004832
    RMS:               0.33304096696301505
Field name for data  1 :  temperature
    Radius:            146 1.53846153846154
    Degree and order:  2 0
    Average:           -0.019162479626085686
    Dtd. deviation:    0.042181275364692204
    RMS:               0.046329910606562996


In [21]:
flib.fin_tave_picked_sph_spectr_f.restype = c_int
flib.fin_tave_picked_sph_spectr_f.argtypes = []

ierr = flib.fin_tave_picked_sph_spectr_f()

## Check obtained data (optional)
Check time average (t_ave_[file_name]), root mean square (t_rms_[file_name]), and standard deviation (t_sigma_[file_name])

In [10]:
ls

picked_mode.dat                 t_rms_icked_mode_l2_m0c.dat
picked_mode_l2_m0c.dat          t_rms_picked_mode_l2_m0c.dat
picked_mode_l2_m0c.dat.gz       t_sigma_icked_mode_l2_m0c.dat
t_ave_icked_mode_l2_m0c.dat     t_sigma_picked_mode_l2_m0c.dat
t_ave_picked_mode_l2_m0c.dat


In [11]:
average_file = "t_ave_" + file_name
f = open(average_file, 'r', encoding='UTF-8')
data = f.read()
print(data)
f.close()

#
# num_layers, num_spectr
               5               1              50
# number of component
              46
t_step    time    radius_ID    radius    degree    order    velocity_pol    velocity_tor    velocity_pol_dr    temperature    pressure    vorticity_pol    vorticity_tor    vorticity_pol_dr    magnetic_field_pol    magnetic_field_tor    magnetic_field_pol_dr    current_density_pol    current_density_tor    current_density_pol_dr    grad_temp_pol    grad_temp_tor    grad_temp_pol_dr    buoyancy_flux    Lorentz_work    vecp_induction_pol    vecp_induction_tor    vecp_induction_pol_dr    rot_Lorentz_force_pol    rot_Lorentz_force_tor    rot_Lorentz_force_pol_dr    rot_buoyancy_pol    rot_buoyancy_tor    rot_buoyancy_pol_dr    rot_Coriolis_force_pol    rot_Coriolis_force_tor    rot_Coriolis_force_pol_dr    heat_flux_pol    heat_flux_tor    heat_flux_pol_dr    inertia_pol    inertia_tor    inertia_pol_dr    Lorentz_force_pol    Lorentz_force_tor    Lorentz_force_pol_dr    rest_o

In [12]:
average_file = "t_rms_" + file_name
f = open(average_file, 'r', encoding='UTF-8')
data = f.read()
print(data)
f.close()

#
# num_layers, num_spectr
               5               1              50
# number of component
              46
t_step    time    radius_ID    radius    degree    order    velocity_pol    velocity_tor    velocity_pol_dr    temperature    pressure    vorticity_pol    vorticity_tor    vorticity_pol_dr    magnetic_field_pol    magnetic_field_tor    magnetic_field_pol_dr    current_density_pol    current_density_tor    current_density_pol_dr    grad_temp_pol    grad_temp_tor    grad_temp_pol_dr    buoyancy_flux    Lorentz_work    vecp_induction_pol    vecp_induction_tor    vecp_induction_pol_dr    rot_Lorentz_force_pol    rot_Lorentz_force_tor    rot_Lorentz_force_pol_dr    rot_buoyancy_pol    rot_buoyancy_tor    rot_buoyancy_pol_dr    rot_Coriolis_force_pol    rot_Coriolis_force_tor    rot_Coriolis_force_pol_dr    heat_flux_pol    heat_flux_tor    heat_flux_pol_dr    inertia_pol    inertia_tor    inertia_pol_dr    Lorentz_force_pol    Lorentz_force_tor    Lorentz_force_pol_dr    rest_o

In [ ]:
average_file = "t_sigma_" + file_name
f = open(average_file, 'r', encoding='UTF-8')
data = f.read()
print(data)
f.close()